In [2]:
import pydicom as pydcm
import matplotlib.pyplot as plt
import pandas as pd
import sys
from pydicom.datadict import dictionary_VR
from pydicom import dcmread,Dataset
from pydicom.filewriter import dcmwrite
import numpy as np
#import cv2
import random
import time
import datetime


from google.cloud import storage
from io import BytesIO
client = storage.Client()

In [3]:
def blob_to_image(bucket, blob_name):
    blob = bucket.get_blob(blob_name)
    blobstring = blob.download_as_string()
    dicomfile = BytesIO(blobstring)
    dicomread = dcmread(dicomfile)
    #plt.imshow(dicomread.pixel_array)
    return dicomread

def img_diff(raw, deid):
    diff = raw.pixel_array-deid.pixel_array
    result = np.count_nonzero(diff)
    return(result)    

def find(list,thing): 
    "finds indices of something in a list"
    return [i for i, j in enumerate(list) if j == thing]


def blob_to_df(filelist, bucket):
    SOP = []
    patient = []
    study = []
    series = []
    DICOM = []
    for i in range(len(filelist)):
        dicom = blob_to_image(bucket, filelist[i])
        DICOM.append(dicom)
        patient.append(dicom.PatientID)
        study.append(dicom.StudyInstanceUID)
        series.append(dicom.SeriesInstanceUID)
        SOP.append(dicom.SOPInstanceUID)
    
    d = {'dicom': DICOM, 'patientID': patient, 'studyID': study, 'seriesID': series, 
         'SOPID': SOP}
    df = pd.DataFrame(d)
    return(df)


In [4]:
bucket_in = client.get_bucket('midi-import')

blobs_in = bucket_in.list_blobs(prefix='ds1_1-600/group2/3470831496')
filelist_in = [item.name for item in blobs_in] #extracting file names from blobs

df = blob_to_df(filelist_in[1:], bucket_in)
#df_in = blob_to_df(filelist_in[1:], bucket_in)

/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.4.637.1.1.0286981.2.191.3459773422647729824'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.4.637.1.1.0286981.2.191.1893866593186486911'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.4.637.1.1.0286981.2.191.2611140455051373536'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.4

In [17]:
#keep list, copied from GCP script keep list
keeplist = ['(0009, 1008)','(7005, 1008)','(7005, 100b)','(7005, 100e)','(7005, 100f)','(7005, 1012)','(7005, 1017)','(7005, 1018)','(7005, 1019)','(7005, 101a)','(7005, 101b)',
               '(7005, 101e)','(7005, 1020)','(7005, 1030)','(0019, 0014)','(0019, 10a3)','(0027, 1033)','(0043, 1035)','(0043, 1036)','(0043, 1037)','(0010, 0040)','(0010, 2203)','(0010, 1020)',
               '(0010, 1030)','(0010, 21c0)','(0010, 21a0)','(0008, 0018)','(0020, 000d)','(0020, 000e)','(0028, 1054)','(2010, 0010)','(0008, 0070)','(0010, 1010)','(0008, 0016)','(0008, 1150)',
               '(0002, 0002)','(0002, 0010)',
               '(0002, 0003)']

In [24]:

import re

file = 0
#vrs to look at
vrs = ['LO', 'SH', 'LT', 'ST', 'UT','IS']
str1 = ''
x = []
x2 = []

patient = []
study = []
SOP = []
series = []
element = []
tag = []
value = []
vr = []
newvalue = []

#regex rules. Finds 'at ' followed by 2-4 capital letters, and 'by ' followed by 2 capital letters
loc = re.compile('at [A-Z]{2,4}')
dr = re.compile('by [A-Z]{2}')

#blacklist. These two strings were just used as a test, wouldn't of been the final blacklist
blacklist = [' MGH', ' ALH']

#loops through each dicom file
for i in range(len(df)):
    file = df.iloc[i][0]
    #loops through each element in the dicom file
    for elem in file.iterall():
        #ignores elements not of VR 'LO', 'SH', 'LT', 'ST', 'UT','IS'
        #ignores Code Value, Private Creator, Actual Frame Duration, and Primary Counts Accumulates elements
        if elem.VR in vrs and elem.name != 'Code Value' and elem.name != 'Private Creator' and elem.name != 'Actual Frame Duration' and elem.name != 'Primary (Prompts) Counts Accumulated':
            #ignores elements specified to be automatically kept by the GCP script (in accordance with the dicom standard)
            if elem.tag not in keeplist:
                str1 = str(elem.value)
                try:
                    #replaces any matches to location or initial regex with *
                    strloc = loc.sub('*',str1)
                    str1 = dr.sub('*',strloc)
                    
                    #loops through the blacklist
                    for b in blacklist:
                        #if a blacklisted string is in the element, replaces it with +
                        if b in str1:
                            str1 = str1.replace(b,'+')
                    #updates the element with new values
                    elem.value = str1
                    
                    
                    #date/pid finder
                    #looks for numeric strings of length 6-15
                    x = re.findall('[0-9]{6,15}', str1)
                    if len(x) != 0:
                        #creating df to look into results. not necessary for de-id. just used for checking results
                        patient.append(df.iloc[i][1])
                        study.append(df.iloc[i][2])
                        series.append(df.iloc[i][3])
                        SOP.append(df.iloc[i][4])
                        element.append(str(elem))
                        tag.append(elem.name)
                        vr.append(elem.VR)
                        value.append(str(elem.value))
                        
                        #loop through each long str of integers
                        for n in x:
                            str1 = str(elem.value)
                            #date format change if numeric str length is 8 and elem vr isn't IS
                            if len(n) == 8 and elem.VR != 'IS':
                                #looks for date of format yyyymmdd and reformats to yyyy mm dd
                                out1 = re.split('(19[0-9][0-9]|20[0-2][0-9])(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])',n)
                                dt1 = ' '.join(out1)
                                
                                #looks for date of format ddmmyyyy and reformats to dd mm yyyy
                                out2 = re.split('(0[1-9]|1[0-9]|2[0-9]|3[0-1])(0[1-9]|1[0-2])(19[0-9][0-9]|20[0-2][0-9])',dt1)
                                dt2 = ' '.join(out2)
                                
                                #looks for date of format mmddyyyy and reformats to mm dd yyyy
                                out3 = re.split('(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])(19[0-9][0-9]|20[0-2][0-9])',dt2)
                                dt3 = ' '.join(out3)
                                
                                #if no date matches were found, removes the int string from the element value
                                if len(out1) == 1 and len(out2)==1 and len(out3)==1:
                                    str2 = str1.replace(n,'')
                                    elem.value = str2
                                    
                                #if dates were found, replace the element value with the string containing the reformatted dates
                                else:
                                    str2 = str1.replace(n,dt3)
                                    elem.value = str2
                                    
                            else:
                                #remove int strings not of length 8, and/or IS VRs
                                str2 = str1.replace(n,'')
                                elem.value = str2
                                
                        #add new val to df. again not important for de-id, just for checking results
                        newvalue.append(str(elem.value))
                                
                except Exception as error:
                    print(str(elem)+'the error:  '+str(error))
                    

In [127]:
#creates df for checking results
d = {'patient':patient, 'study':study, 'series':series, 'SOP':SOP, 'tag':tag, 'vr':vr, 'element':element, 'value':value, 'newvalue':newvalue}
flagged = pd.DataFrame(d)

In [9]:
#uploads changed dicom files to gcp
bucket = client.get_bucket('midi-import')
for i in range(1,len(filelist_in)):
    filepath = 'preprocessed/ds1_1-23preprocessed6-27/'+filelist_in[i]
    blob = bucket.blob(filepath)
    df.iloc[i-1][0].save_as('test6.dcm')
    blob.upload_from_filename('test6.dcm')
    